In [ ]:
from IPython.display import YouTubeVideo, Markdown, SVG, Code
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Acelerando Python con Cython

La [implementación](https://wiki.python.org/moin/PythonImplementations) más utilizada del interprete/compilador de Python está escrita en C y se llama [CPython](https://github.com/python/cpython). Recordemos que la implementación es aquello que interpreta y corre el código escrito en Python. Existen otras implementaciones de Python menos usadas como [PyPy](https://www.pypy.org/), [Jython](https://www.jython.org/) o IronPython. 

CPython compila el código escrito en Python en un código de máquina (binario) de forma transparente al usuario. Luego CPython interpreta el binario. Si nos interesa, podemos estudiar dicho código binario usando el módulo [dis](https://docs.python.org/3.7/library/dis.html) de Python

Luego, si sabemos C podemos usar la [API de Python/C](https://docs.python.org/3/c-api/index.html) para 

1. Escribir módulos de C que puedan llamarse desde Python
1. Hacer interfaces entre código C y Python

Sin embargo la API es un poco complicada y existen alternativas menos tediosas para lograr estos objetivos, como por ejemplo

- [ctypes](https://docs.python.org/2/library/ctypes.html) 
- [CFFI](https://cffi.readthedocs.io/en/latest/)
- [SWIG](http://swig.org/)
- Cython

En esta lección veremos en detalle esta última

## [Cython](https://cython.org/): C Extensions for Python

Recordemos que **Python** es un lenguaje interpretado con tipos dinámicos. Esto hace que cada operación tenga un overhead. Por ejemplo

```python
    z = x + y
    # overhead: Inferir el tipo de x
    # overhead: Inferir el tipo de y
    # Hacer la operación suma
    # overhead: Darle el tipo adecuado a z
```

en cambio, **Cython** es un lenguaje de programación que le agrega a Python algunas propiedades de C y C++, una de ellas son los **tipos estáticos**

```python
    int x = 1
    int y = 2
    int z = x + y
    # No hay que inferir el tipo de x, y, z
```

Esto hace que Cython sea menos flexible pero decenas de veces más rápido que Python. Además en términos de sintaxis Cython es casi tan simple como Python. Sin embargo a diferencia de Python, el lenguaje Cython debe compilarse

- El compilador de Cython convierte el código fuente en código C 
- Luego el código C se compila como un módulo de Python con la implementación **CPython**

En pocas palabras, una vez compilado el código escrito en Cython este puede llamarse desde Python!

¿Por qué considerar Cython?

- Cython es casi tan simple como Python y casi tan rápido como C
- Con Cython se pueden llamar funciones y librerías de C
- Cython se integra de buena manera con NumPy

Por ende Cython es muy atractivo para proyectos que usen Python y tengan requisitos de alto rendimiento. Estudiemos la sintaxis de Cython mediante algunos ejemplos

A lo largo de esta lección usaremos como ejemplo el cálculo de la **Distancia euclidiana todos-con-todos**. Sea un conjunto de $N=1000$ datos bidimensionales ($D=2$) donde queremos calcular la distancia euclidiana de cada dato con todos los demás, es decir una matriz donde el elemento $ij$ es 

$$
\text{dist}_{ij} = \sqrt {\sum_{k=1}^D (x_{ik} - x_{jk})^2}
$$

A continuación se muestran dos códigos que cumplen este proposito y obtienen un resultado equivalente

- El primero usa "Python puro" y calcula las distancias de forma secuencial
- El segundo usa operaciones vectoriales de NumPy y calcula las distancias "al mismo tiempo"

Estudie ambos códigos hasta comprenderlos

In [ ]:
data = np.random.randn(1000, 2)

def distancia_pares(data):    
    N, D = data.shape
    dist = np.empty(shape=(N, N))
    for i in range(N):
        for j in range(i, N):
            dist[i, j] = 0
            for k in range(D):
                dist[i, j] += (data[i, k] - data[j, k])**2
            dist[i, j] = np.sqrt(dist[i, j])
            dist[j, i] = dist[i, j]
    return dist
            
def distancia_pares_numpy(data):
    return np.sqrt(np.sum((data.reshape(-1, 1, 2) - data.reshape(1, -1, 2))**2, axis=-1))

np.allclose(distancia_pares(data), distancia_pares_numpy(data))

In [ ]:
%timeit -r3 -n1 distancia_pares(data)

In [ ]:
%timeit -r3 -n1 distancia_pares_numpy(data)

## Cython desde IPython

Primero debemos instalar cython en nuestro ambiente de conda

    conda install cython

Luego en IPython podemos cargar la extensión `cython` como se muestra a continuación

In [ ]:
%load_ext cython

con esto tendremos disponible la magia de bloque `%%cython`

Un bloque donde se use esta magia acepta lenguaje cython y se compila al ejecutarlo. Luego podremos llamar las funciones de ese bloque desde bloques regulares de Python

Si surgen errores de compulación estos aparecen como la salida del bloque. Notar que este bloque está "desconectado" del resto del notebook, por lo que debe tener sus propios `import`

La magia tiene las siguientes opciones

- `-a` (annotate) retorna un profile linea a linea indicando con amarillo las llamadas a CPython (mientras más llamadas más lento es nuestro código)
- `-+` Usar C++ en lugar de C
- `-c` Argumentos de compilación
- `-l` librerías para linkear a nuestro código
- `-L` directorio con librerías
- `-I` directorio con cabeceras (include)   


Veamos que ocurre al agregarle la magia al ejemplo anterior

In [ ]:
%%cython

import numpy as np

def distancia_pares_cython_inocente(data):    
    N, D = data.shape
    dist = np.empty(shape=(N, N))
    for i in range(N):
        for j in range(i, N):
            dist[i, j] = 0.0
            for k in range(D):
                dist[i, j] += (data[i, k] - data[j, k])**2
            dist[i, j] = np.sqrt(dist[i, j])
            dist[j, i] = dist[i, j]
    return dist

In [ ]:
np.allclose(distancia_pares(data), distancia_pares_cython_inocente(data))

In [ ]:
%timeit -r3 -n1 distancia_pares(data)

In [ ]:
%timeit -r3 -n1 distancia_pares_numpy(data)

In [ ]:
%timeit -r3 -n1 distancia_pares_cython_inocente(data)

El resultado es idéntico pero como no hemos hecho ningún cambio el tiempo de ejecución es sólo levemente mejor que la versión en Python puro

A continuación veremos como "cythonizar" nuestro código para ganar rendimiento

## Mejora 1: Definiendo tipos en Cython

En Cython se definen tipos estáticos con el keyword `cdef` seguido del tipo y luego el nombre. Por ejemplo una variable de tipo `double` llamada `mi_variable`:

```cython
cdef double mi_variable 
```
    
Para los arreglos (ndarray) se usan ["memory-views"](https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html#memoryviews). Por ejemplo un *memory-view* para una arreglo de tres dimensiones:

```cython
cdef double [:, :, :] mi_arreglo
```    

Y se puede ganar un poco más de rendimiento usando el operador `::1` para especificar si el arreglo es *row-major* (estilo C)

```cython
cdef double [:, :, ::1] mi_arreglo
```    

o *column-major* (estilo Fortran)

```cython
cdef double [::1, :, :] mi_arreglo
```

Veamos como queda el ejemplo introduciendo tipos estáticos

In [ ]:
%%cython 

import numpy as np

def distancia_pares_cython_estatico(double [:, ::1] data):
    # Definimos el tipo de N, D, dist y data
    cdef int N = data.shape[0]
    cdef int D = data.shape[1]
    dist = np.empty(shape=(N, N), dtype=np.double)
    cdef double [:, ::1] dist_view = dist
    # También definimos los índices, se puede usar int o Py_ssize_t 
    cdef int i, j, k
    for i in range(N):
        for j in range(i, N):
            dist_view[i, j] = 0.0
            for k in range(D):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            dist_view[i, j] = np.sqrt(dist_view[i, j])
            dist_view[j, i] = dist_view[i, j]
    return dist

In [ ]:
np.allclose(distancia_pares(data), distancia_pares_cython_estatico(data))

In [ ]:
%timeit -r3 -n1 distancia_pares_numpy(data) 

In [ ]:
%timeit -r3 -n1 distancia_pares_cython_estatico(data)

Con solo definir el tipo de data, dist, $N$, $D$ y los índices hemos obtenido un *speed-up* importante con respecto a "Python puro" aunque sigue siendo más lento que NumPy

Si observaramos el código anotado por cython y revisemos la cantidad de llamadas a CPython de cada linea notaríamos que la linea 17 es particularmente conflictiva 

- involucra una gran cantidad de instrucciones
- se llama NxN veces

esto se debe a que estamos usando la función de NumPy `np.sqrt`. Podemos obtener un rendimiento mucho mejor si usamos la implementación de raíz cuadrada de C

## Mejora 2: Llamando funciones de C desde Cython

Es posible llamar funciones de C desde Cython de forma sencilla. Consideremos como ejemplo la función `sqrt` de la [librería matemática estándar de C](http://pubs.opengroup.org/onlinepubs/9699919799/basedefs/math.h.html). 

Necesitamos dos para usar `sqrt` desde cython:

- Importar la función `sqrt` desde la cabecera `math.h`: Esto se hace con el keyword `cdef extern from`
- Compilar contra `libm`

Veamos como queda nuestro ejemplo con esta modificación

In [ ]:
%%cython -l m

import numpy as np

cdef extern from "math.h":
    double sqrt(double)

def distancia_pares_cython_sqrtC(double [:, ::1] data):
    # Definimos el tipo de N, D, dist y data
    cdef int N = data.shape[0]
    cdef int D = data.shape[1]
    dist = np.empty(shape=(N, N), dtype=np.double)
    cdef double [:, ::1] dist_view = dist
    # También definimos los índices, se puede usar int o Py_ssize_t 
    cdef Py_ssize_t i, j, k
    for i in range(N):
        for j in range(i, N):
            dist_view[i, j] = 0.0
            for k in range(D):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            dist_view[i, j] = sqrt(dist_view[i, j])
            dist_view[j, i] = dist_view[i, j]
    return dist

In [ ]:
np.allclose(distancia_pares(data), distancia_pares_cython_sqrtC(data))

In [ ]:
%timeit -r3 -n1 distancia_pares_numpy(data) 

In [ ]:
%timeit -r3 -n1 distancia_pares_cython_estatico(data)

In [ ]:
%timeit -r3 -n1 distancia_pares_cython_sqrtC(data)

Con esta simple modificación hemos obtenido un tiempo de ejecución incluso menor que la implementación en NumPy, excelente!

Es **sumamente importante** notar que usando `extern` podemos hacer interfaces [entre Python y casi cualquier código escrito en C](https://cython.readthedocs.io/en/latest/src/tutorial/clibraries.html)

## Mejora 3: Deshabilitando comprobaciones para ir aun más rápido

Podemos hacer nuestro código más rápido (y más inseguro) deshabilitando las verificaciones que Python realiza por defecto

En Cython esto se logra usando decoradores que funcionan como directivas de compilación. Las opciones disponibles se encuentrán [aquí](https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives). En este caso particular deshabilitaremos dos verificaciones: *boundscheck* y *wraparound*

Al deshabilitarlas el código no comprobará si escribimos fuera del arreglo y tampoco traducirá índices negativos

También aprovecharemos de hacer un cambio menor que nos será de utilidad más adelante:

- Definiremos el tipo de data y dist de forma más conveniente usando `ctypedef`
- Para esto incluiremos un modulo de Cython llamado numpy usando `cimport` que contiene definiciones tipo C  


In [ ]:
%%cython -l m
import cython
cimport numpy as npc
import numpy as np

# Por conveniencia podemos definir el tipo de data y dist con 
ctypedef npc.float64_t TIPO_t
TIPO = np.float64

cdef extern from "math.h":
    TIPO_t sqrt(TIPO_t)

# Deshabilitamos las comprobaciones de Python:
@cython.boundscheck(False)
@cython.wraparound(False)
def distancia_pares_cython(TIPO_t [:, ::1] data):
    cdef int N = data.shape[0]
    cdef int M = data.shape[1]
    dist = np.empty(shape=(N, N), dtype=TIPO)
    cdef TIPO_t [:, ::1] dist_view = dist
    cdef Py_ssize_t i, j, k
    for i in range(N):
        for j in range(i, N):
            dist_view[i, j] = 0.0
            for k in range(M):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            dist_view[i, j] = sqrt(dist_view[i, j])
            dist_view[j, i] = dist_view[i, j]
    return dist

In [ ]:
np.allclose(distancia_pares(data), distancia_pares_cython(data))

In [ ]:
%timeit -r3 -n1 distancia_pares_cython_sqrtC(data)

In [ ]:
%timeit -r3 -n1 distancia_pares_cython(data)

Un leve speed-up "casi gratis"

##  Mejora 4: Mayor flexibilidad con tipos de Cython fusionados 

En este momento es importante recordar lo que hemos perdido al usar Cython con respecto a Python: **Flexibilidad**

Si usamos un argumento que no sea `double` nuestra función en Cython retornará un error. Es nuestra responsabilidad evitar que esto ocurra

In [ ]:
data_float32 = data.astype(np.float32)
distancia_pares_cython(data_float32)

Podemos resolver este problema usando los tipos fusionados de Cython. Si queremos fusionar dos tipos de datos "tipo1" y "tipo2" bajo el nombre "tipo3", lo escribimos usando

```cython
ctypedef fused tipo3:
    tipo1
    tipo2
```

De forma transparente Cython creará dos funciones en lugar de una. 

Modifiquemos la función `distancia_pares_cython` para que acepte los tipos double e float como un tipo fusionado. En este caso no debemos olvidar importar la definición de `sqrtf` de `math.h`

In [ ]:
%%cython -l m
import cython
cimport numpy as npc
import numpy as np

# Tipo fusionado
ctypedef fused TIPO_t:
    npc.float32_t
    npc.float64_t # double
    
cdef extern from "math.h":
    npc.float32_t sqrtf(npc.float32_t) #Definición para float32
    npc.float64_t sqrt(npc.float64_t) # Definición para float64

@cython.boundscheck(False)
@cython.wraparound(False)
def distancia_pares_cython_multitipo(TIPO_t [:, ::1] data):
    cdef int N = data.shape[0]
    cdef int M = data.shape[1]
    # Comprobamos el tipo antes de crear el arreglo de numpy
    if TIPO_t is npc.float32_t:
        TIPO = np.float32
    else:
        TIPO = np.float64
    dist = np.empty(shape=(N, N), dtype=TIPO)
    cdef TIPO_t [:, ::1] dist_view = dist
    cdef Py_ssize_t i, j, k
    for i in range(N):
        for j in range(N):
            dist_view[i, j] = 0.0
            for k in range(M):
                dist_view[i, j] += (data[i, k] - data[j, k])**2
            if TIPO_t is npc.float32_t:
                dist_view[i, j] = sqrtf(dist_view[i, j])
            else:
                dist_view[i, j] = sqrt(dist_view[i, j])
    return dist

In [ ]:
np.allclose(distancia_pares_numpy(data),
            distancia_pares_cython_multitipo(data))

In [ ]:
np.allclose(distancia_pares_numpy(data_float32),
            distancia_pares_cython_multitipo(data_float32))

In [ ]:
%timeit -r3 -n1 distancia_pares_numpy(data_float32)

In [ ]:
%timeit -r3 -n1 distancia_pares_cython_multitipo(data_float32)

El resultado usando `float` es equivalente al de NumPy y la velocidad es similar a la de `distancia_pares_cython`, es decir que la pérdida de rendimiento no es notoria


## Ejemplo formativo: "Cythonizando" el fractal de Julia

Partiendo del código de Python puro del fractal de Julia, escribimos una versión en Cython

Recordemos:

- Usamos `cdef` para definir las variables con tipo estático
    - También podemos usar `cdef` para definir funciones con tipo
    - Las funciones con tipo solo pueden ser llamadas por otras funciones de Cython
- Usamos `ctypedef` para definir tipos 
- Usamos `cimport` para importar otros módulos de Cython
- Usamos memory-views con el tipo adecuado para conectar con NumPy en la salida
- Deshabilitamos las comprobaciones de Python para ganar más velocidad


In [ ]:
%%cython 
import cython
cimport numpy as npc
import numpy as np

ctypedef npc.float64_t TIPOF_t
ctypedef npc.int64_t TIPOI_t

# Las funciones con cdef solo pueden ser llamadas desde Cython
cdef TIPOI_t evaluate_z(TIPOF_t zi, TIPOF_t zr, int maxiters=50, TIPOF_t cr=-0.835, TIPOF_t ci=-0.2321):
    cdef:
        TIPOI_t nit = 0
        TIPOF_t zi2 = zi**2
        TIPOF_t zr2 = zr**2
        
    while zi2 + zr2 <= 4. and nit < maxiters:
        zi = 2.*zr*zi + ci
        zr = zr2 - zi2 + cr
        zr2 = zr**2
        zi2 = zi**2 
        nit +=1
    return nit

# Las funciones con def pueden ser llamadas desde Python
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def make_fractal_cython(int N, int maxiters=50):
    image = np.empty(shape=(N, 2*N), dtype=np.int64)
    cdef TIPOI_t [:, ::1] image_view = image
    cdef:
        Py_ssize_t i, j
    # Los ndarray no se copian, los podemos modificar inplace desde Cython
    for i in range(N):
        for j in range(2*N):
            image_view[i, j] = evaluate_z(-1.+i*2./N, -2.+j*2./N, maxiters)
    return image

In [ ]:
from fractal import make_fractal
N = 500
np.allclose(make_fractal(N), make_fractal_cython(N))

In [ ]:
%timeit -r3 -n1 make_fractal(N)

In [ ]:
%timeit -r3 -n1 make_fractal_cython(N)

El resultado es idéntico al código secuencial en "Python puro" y el tiempo es considerablemente menor

Con esto podemos dibujar el fractal en mayor resolución en un tiempo razonable :)

In [ ]:
N = 4000
plt.figure(figsize=(8, 4), tight_layout=True)
plt.gca().axis('off')
plt.imshow(make_fractal_cython(N));

**Propuesto:** 

Dibuje curvas de *speed-up* para el código en cython vs "Python puro" considerando $N=10, 50, 100, 500, 1000, 5000$

## Tópicos adicionales

Lecturas sobre temas específicos de Cython:

- [Incluyendo código Cython en un módulo de Python](https://cython.readthedocs.io/en/latest/src/quickstart/build.html#building-a-cython-module-using-distutils)
- [Haciendo profiling de código Cython](https://cython.readthedocs.io/en/latest/src/tutorial/profiling_tutorial.html)
- [Malloc/Free en Cython](https://cython.readthedocs.io/en/latest/src/tutorial/memory_allocation.html)
- Libro [Cython: A Guide for Python Programmers](https://pythonbooks.org/cython-a-guide-for-python-programmers/)